In [15]:
import rootpath
import pandas as pd
from joblib import Parallel , delayed
import numpy as np

In [2]:
root = rootpath.detect()
root

'/home/yiyi/Documents/masterarbeit/CPD'

In [3]:
import os
data_dir = os.path.join(root, 'data/sentiment_analysis/results')

In [4]:
spellchecked_dir = os.path.join(root, 'data/spellchecked')

In [5]:
data_dir

'/home/yiyi/Documents/masterarbeit/CPD/data/sentiment_analysis/results'

In [6]:
count =0 
filename = '422#793f568b-3393-4e81-a940-6b2a84ca44bb'

filepath = os.path.join(data_dir, filename)

In [7]:
renovations = [
    'build',
    'carpeting',
    'caulking',
    'centerpiece',
    'construction',
    'damage',
    'date',
    'dated',
    'decor',
    'dire',
    'décor',
    'exterior',
    'facelift',
    'fix',
    'furnish',
    'furnishing',
    'granite',
    'grout',
    'improvement',
    'maintenance',
    'makeover',
    'modernisation',
    'outdated',
    'overhaul',
    'paint',
    'painting',
    'plumbing',
    'reconstruction',
    'redecorating',
    'redo',
    'redone',
    'refurb',
    'refurbish',
    'refurbishment',
    'remodel',
    'remodeling',
    'reno',
    'renovate',
    'renovated',
    'renovation',
    'repair',
    'replace',
    'replacement',
    'reset',
    'rust',
    'spruce',
    'stylish',
    'tape',
    'tlc',
    'update',
    'updating',
    'upgrading',
    'worn']

In [8]:
from gensim.models import Word2Vec


unable to import 'smart_open.gcs', disabling that module


In [9]:
import enchant
from spellchecker import SpellChecker

In [12]:
spell= SpellChecker()

In [10]:
df = pd.read_csv(filepath)

In [11]:
df.head(2)

,uid,lemma,sentence,sentiment,date,score
0,3a68d8ea-6f20-48a2-a3a0-d0eb4a4c04f8,read review book hotel come prepare disappoint...,I read the reviews before booking this hotel a...,-0.117656,2008-09-01 00:00:00,40.0
1,3a68d8ea-6f20-48a2-a3a0-d0eb4a4c04f8,room small,The room was extremely small,-0.487555,2008-09-01 00:00:00,40.0


In [16]:
df['date'] = df['date'].astype('datetime64')
df = df[df['date']>= np.datetime64("2015-01-01")]
df['lemma']= df['lemma'].astype(str)
lemmas = [lemma.lower().split() for lemma in df['lemma']]

In [18]:
len(lemmas)

4199

In [19]:
# corrected_lemmas= Parallel(n_jobs=-1)(delayed(spell_check)(x) for x in lemmas)
corrected_lemmas = [[spell.correction(lemma) for lemma in x] for x in lemmas]


In [20]:
model = Word2Vec(corrected_lemmas,size=200, window=6, min_count=2, workers=8, iter=200, negative=5)

In [21]:
model.most_similar('suite')

/home/yiyi/anaconda3/envs/cdp/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('bedroom', 0.6694496870040894),
 ('gross', 0.5565172433853149),
 ('hotelscom', 0.5559185743331909),
 ('pristine', 0.5384526252746582),
 ('setup', 0.5165766477584839),
 ('appoint', 0.5075203776359558),
 ('picture', 0.5032559633255005),
 ('reserve', 0.49055930972099304),
 ('upgrade', 0.4787270426750183),
 ('rust', 0.47131267189979553)]

In [22]:
model.wv.vectors.shape[0]

1458

In [23]:
count =0 
for filename in os.listdir(data_dir):
    filepath = os.path.join(data_dir, filename)
    df = pd.read_csv(filepath)
    output_path = os.path.join(spellchecked_dir, filename)
    
    df['date'] = df['date'].astype('datetime64')
    df = df[df['date']>= np.datetime64("2015-01-01")]
    df['lemma']= df['lemma'].astype(str)
    lemmas = [lemma.lower().split() for lemma in df['lemma']]
    # lemmas_corrected= Parallel(n_jobs=-1)(delayed(spell_check)(x) for x in lemmas)
    lemmas_corrected = [[spell.correction(lemma) for lemma in x] for x in lemmas]
    
    df['lemma'] = lemmas_corrected
    df.to_csv(output_path)
    print('save to_', output_path)
    model.build_vocab(lemmas_corrected, update=True)
    model.train(lemmas_corrected, total_examples= len(lemmas_corrected), epochs =2)
    count+=1

save to_ /home/yiyi/Documents/masterarbeit/CPD/data/spellchecked/267#0ed315c4-9b7b-4423-a320-04cdca7d97ad


KeyboardInterrupt: 

In [ ]:
count

In [ ]:
model.wv.vocab

In [ ]:
keywords = [['room','suite', 'minibar', 'bathroom', 'view', 'shower', 'wifi', 
             'balcony', 'tv', 'furniture', 'bathtub'],
            ['restaurant', 'menu', 'dinner', 'breakfast', 'lunch', 
             'bar', 'café','dining', 'dish','snack', 'dessert'],
            ['facility', 'amenity', 'elevator', 'lobby'],
            ['pool','swim'],
            ['family', 'vacation'],
            ['price', 'cost'],
            ['atmosphere', 'ambience', 'vibe', 'experience'],
            ['fitness', 'spa', 'sport', 'golf', 'gym'],
            ['transport', 'shuttle', 'location'],       
            ['beach', 'sea', 'ocean', 'lake'], 
            ['casino', 'entertainment'],
            ['venue', 'meeting', 'event', 'ballroom'],
            [ 'parking', 'park'],
            ['reception', 'service', 'staff', 'concierge'],
            [ 'renovation','revamp', 'overhaul', 'remodel', 'update', 'redone', 'redo',
                 'remodel', 'renovate', 'redecorate',
                 'refurbish', 'repair', 'refit', 'recondition',
                 'renew', 'renewal', 'reform']]

In [26]:
reno_keywords = ['revamp', 'overhaul', 'remodel', 'update', 'redone', 'redo',
                 'remodel', 'renovation', 'renovate', 'redecorate',
                 'refurbish', 'repair', 'refit', 'recondition',
                 'renew', 'renewal', 'reform']

In [27]:
renos =[]
for word in reno_keywords:
    try:
        renos.append(list([x for x,y in model.most_similar(word, topn=20 )]))
    except Exception:
        pass

/home/yiyi/anaconda3/envs/cpd/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  after removing the cwd from sys.path.


In [29]:
from itertools import chain
renos_ = list(set(list(chain.from_iterable(renos))))

In [31]:
sorted(renos_)

['1970',
 'anniv',
 'bride',
 'bridesmaid',
 'broke',
 'byli',
 'caulking',
 'construction',
 'dateing',
 'deerbrooke',
 'demolishment',
 'desperatley',
 'engagement',
 'everyrthe',
 'expansion',
 'facelift',
 'fix',
 'furbishment',
 'furnishing',
 'graduate',
 'imporvement',
 'incluinde',
 'liiiiittle',
 'maintainance',
 'maintainence',
 'maintanance',
 'maintance',
 'maintenace',
 'maintenance',
 'maintence',
 'maintenence',
 'makeover',
 'marriage',
 'modernisation',
 'modernise',
 'modernising',
 'modernization',
 'modernize',
 'modernized',
 'nuptial',
 'overhall',
 'overhaul',
 'overhauling',
 'paylate',
 'plumber',
 'plumbing',
 'proposal',
 'reconstruction',
 'redecorate',
 'redecorating',
 'redecoration',
 'redeedme',
 'redesign',
 'redo',
 'redone',
 'referb',
 'refinishing',
 'refit',
 'refresh',
 'refurb',
 'refurbe',
 'refurbish',
 'refurbished',
 'refurbishing',
 'refurbishment',
 'refurnish',
 'reglazing',
 'relifting',
 'remodel',
 'remodelation',
 'remodeling',
 'remod

In [22]:
from collections import defaultdict

keywords_similar = defaultdict(list)
for words in keywords:
    keyword= words[0]
    for word in words:
        try:
            keywords_similar[keyword].append(list([x for x,y in model.most_similar(word, topn=20 )]))
        except Exception:
            pass
    keywords_similar[keyword].append(words)

/home/yiyi/anaconda3/envs/cpd/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


In [32]:
model.most_similar("noise", topn=20)

/home/yiyi/anaconda3/envs/cpd/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('noice', 0.8472913503646851),
 ('sound', 0.6614988446235657),
 ('noisy', 0.6463830471038818),
 ('banging', 0.5855908393859863),
 ('ruckus', 0.5773318409919739),
 ('audible', 0.5717959403991699),
 ('loud', 0.5664687156677246),
 ('rumble', 0.5656085014343262),
 ('noisey', 0.5468951463699341),
 ('jackhammering', 0.5466371178627014),
 ('commotion', 0.5412279367446899),
 ('hum', 0.5396388173103333),
 ('disturbance', 0.5368995666503906),
 ('thumping', 0.533816933631897),
 ('disruptive', 0.5334421396255493),
 ('thud', 0.5293156504631042),
 ('siren', 0.5273467302322388),
 ('pounding', 0.5225614905357361),
 ('humming', 0.518813967704773),
 ('drilling', 0.5138258934020996)]

In [24]:
from itertools import chain

keywords_similars = {}
for keyword, value in keywords_similar.items():
    keywords_similars[keyword]=sorted(list(set(list(chain.from_iterable(value))))) 

In [25]:
import json
with open('keywords.json', 'w') as file:
    json.dump(keywords_similars, file)

In [19]:
import json
with open('keywords.json', 'w')as file:
    json.dump(keywords_similar, file)

In [30]:
aspects = []
for key, value in keywords_similar.items():
    aspects.append(key)
    aspects+= value
    

In [ ]:
sorted(list(set(aspects)))

In [34]:
keywords_reno = {}
for word in reno_keywords:
    try:
        keywords_reno[word]=list([x for x,y in model.most_similar(word, topn=20 )])
    except Exception:
        pass

/home/yiyi/anaconda3/envs/cpd/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  after removing the cwd from sys.path.


In [36]:
renowords = []
for key, value in keywords_reno.items():
    renowords.append(key)
    renowords+= value
    

In [ ]:
sorted(list(set(renowords)))